# Reddit Vaccine Myth Sentiment Analysis

According to the description of this dataset, r/VaccineMyths is a subreddit where myths related to various vaccines are discussed.

It contains both posts and comments.

Our task is to perform sentiment analysis and then topic modeling on this dataset.
The topic modeling will be particularly helpful for the medical study of this dataset, because it will let us better understand what are the factors that are related to the originating of particular types of myths surrouding the vaccines.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
dataset_path = os.path.join(dirname, filenames[0])

In [3]:
# Class for decorated printing
# Source: https://stackoverflow.com/a/17303428
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [4]:
# Read data
df = pd.read_csv(dataset_path)
# Print top 5 rows
df.head()

### Exploratory Data Analysis

I have written some reusable utility functions for the EDA

In [5]:
# Shape of dataset
print(f'Shape of dataset: {df.shape}')

In [6]:
def basic_information(df):
    print(color.PURPLE + color.BOLD + 'Description: \n' + color.END)
    display(df.describe())
    print(color.PURPLE + color.BOLD + '\nInformation: \n' + color.END)
    display(df.info())
    print(color.PURPLE + color.BOLD + '\nNaN Values: \n' + color.END)
    display(pd.isna(df).sum())

In [7]:
def feature_type(df):
    '''
    This function finds which features in given dataframe 
    are of categorical type and which are numerical
    
    Input - df:Dataframe
    Output - categorical_features:list, numerical_features:list 
    '''
    categorical_features = []
    numerical_features = []
    
    for column in df.columns:
        if df[column].dtype == object:
            categorical_features.append(column)
        else:
            numerical_features.append(column)
            
    return categorical_features, numerical_features

In [21]:
basic_information(df)

In [9]:
categorical_features, numerical_features = feature_type(df)
print(color.PURPLE + color.BOLD + 'Categorical Features: ' + color.END + f'{categorical_features}')
print(color.PURPLE + color.BOLD + 'Categorical Features: ' + color.END + f'{categorical_features}')

From here, we can see the basic statistical details of the dataset
We can observe that the columns **url** and **body** contain null data.
Also the dataset contains categorical as well as numerical features.

## Visualizing patterns in the occurrence of missing values

Here we would like to see if there is some pattern in the occurrence of the missing values of a column or not. For eg. Are missing values sparsely localted all over the domain, or just located together somewhere specifically as a chunk.

With a heatmap, we would also be able to observe if there is any positive or negative co-occurrence relation between missing values of different columns.

In [10]:
# Visualizing patterns in the occurrence of missing values
plt.figure(figsize=(7,7))
sns.heatmap(df.isna(), cbar=False)
plt.title("Heatmap of missing data pattern", fontsize=25)

We can observe that for url column, there is a bulk absence of values for a part of the domain. Also, there is somewhat a negative correlation between the missing values of **url** and those of **body**, meaning missing values are present for the part fo domain in **body**, where they are present for **url** and vice-versa.

In [12]:
# Creating month and year indices with the timestamp data of the dataset
df['year'] = pd.DatetimeIndex(df['timestamp']).year
df['month'] = pd.DatetimeIndex(df['timestamp']).month

In [13]:
plt.figure(figsize=(7,7))
sns.heatmap(df.corr(), annot=True)
plt.title("Heatmap of co-occurrence matrix", fontsize=25)

We can see there is a high correlation between **year** and **created**, and **comms_num** and **score**, so one column can be dropped in each pair

In [ ]:
df.drop(labels=['created', 'comms_num', 'url', 'id', 'timestamp'], axis=1, inplace=True)

In [24]:
df.head()